# Notebook camada Gold

In [19]:
import findspark
import os
findspark.init()

from pyspark.sql import SparkSession

# Criar uma sessão do Spark
spark = SparkSession.builder.appName("Notebook_Gold").getOrCreate()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, avg, max, sha2, col, unhex, substring, expr

### Camadas de Origem e Destino

In [20]:
camada_origem = "silver"
camada_destino = "gold"
os.makedirs(camada_origem, exist_ok=True)

### Carregar tabelas da Silver

In [21]:
# Carrega os dados da camada Silver (Parquet)
customers = spark.read.parquet(f"{camada_origem}/customers")
geolocation = spark.read.parquet(f"{camada_origem}/geolocation")
order_items = spark.read.parquet(f"{camada_origem}/order_items")
order_payments = spark.read.parquet(f"{camada_origem}/order_payments")
order_reviews = spark.read.parquet(f"{camada_origem}/order_reviews")
orders = spark.read.parquet(f"{camada_origem}/orders")
product_category_translation = spark.read.parquet(f"{camada_origem}/product_category_name_translation")
products = spark.read.parquet(f"{camada_origem}/products")
sellers = spark.read.parquet(f"{camada_origem}/sellers")

### Dimensão Cliente

In [22]:
# Adiciona uma coluna com UUID gerado nativamente pelo Spark
dim_cliente = customers.select(
    "customer_id",
    "customer_unique_id",
    "customer_zip_code_prefix",
    "customer_city",
    "customer_state"
).withColumn("customer_sk", expr("uuid()"))

# Reordenar as colunas
dim_cliente = dim_cliente.select(
    "customer_sk",
    "customer_id",
    "customer_unique_id",
    "customer_zip_code_prefix",
    "customer_city",
    "customer_state"
)

# Salva a dimensão vendedor na camada Gold
dim_cliente.write.parquet(f"{camada_destino}/dim_cliente", mode="overwrite")


25/02/24 08:33:02 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


### Dimensão Vendedor

In [25]:
# Seleciona as colunas da tabela sellers
dim_vendedor = sellers.select(
    "seller_id",
    "seller_zip_code_prefix",
    "seller_city",
    "seller_state"
).withColumn("seller_sk", expr("uuid()"))  # Adiciona uma chave substituta única

# Reordena as colunas para colocar o seller_sk no início
dim_vendedor = dim_vendedor.select(
    "seller_sk",
    "seller_id",
    "seller_zip_code_prefix",
    "seller_city",
    "seller_state"
)

# Salva a dimensão vendedor na camada Gold
dim_vendedor.write.parquet(f"{camada_destino}/dim_vendedor", mode="overwrite")

### Dimensão Produto

In [26]:
# Adiciona a chave substituta com UUID
dim_produto = products \
    .join(product_category_translation, "product_category_name", "left") \
    .select(
        "product_id",
        "product_category_name",
        "product_category_name_english",
        "product_name_lenght",
        "product_description_lenght",
        "product_photos_qty",
        "product_weight_g",
        "product_length_cm",
        "product_height_cm",
        "product_width_cm"
    ).withColumn("product_sk", expr("uuid()"))  # Adiciona UUID como chave substituta

# Reordena as colunas para colocar a chave substituta no início
dim_produto = dim_produto.select(
    "product_sk",
    "product_id",
    "product_category_name",
    "product_category_name_english",
    "product_name_lenght",
    "product_description_lenght",
    "product_photos_qty",
    "product_weight_g",
    "product_length_cm",
    "product_height_cm",
    "product_width_cm"
)

# Salva a dimensão produto na camada Gold
dim_produto.write.parquet(f"{camada_destino}/dim_produto", mode="overwrite")


### Dimensão Pagamento

In [27]:
# Adiciona a chave substituta com UUID
dim_pagamento = order_payments.select(
    "order_id",
    "payment_sequential",
    "payment_type",
    "payment_installments",
    "payment_value"
).withColumn("payment_sk", expr("uuid()"))  # Adiciona UUID como chave substituta

# Reordena as colunas para colocar a chave substituta no início
dim_pagamento = dim_pagamento.select(
    "payment_sk",
    "order_id",
    "payment_sequential",
    "payment_type",
    "payment_installments",
    "payment_value"
)

# Salva a dimensão pagamento na camada Gold
dim_pagamento.write.parquet(f"{camada_destino}/dim_pagamento", mode="overwrite")

### Dimensão Avaliação

In [28]:
# Adiciona a chave substituta com UUID
dim_avaliacao = order_reviews.select(
    "order_id",
    "review_id",
    "review_score",
    "review_comment_title",
    "review_comment_message",
    "review_creation_date",
    "review_answer_timestamp"
).withColumn("review_sk", expr("uuid()"))  # Adiciona UUID como chave substituta

# Reordena as colunas para colocar a chave substituta no início
dim_avaliacao = dim_avaliacao.select(
    "review_sk",
    "order_id",
    "review_id",
    "review_score",
    "review_comment_title",
    "review_comment_message",
    "review_creation_date",
    "review_answer_timestamp"
)

# Salva a dimensão avaliação na camada Gold
dim_avaliacao.write.parquet(f"{camada_destino}/dim_avaliacao", mode="overwrite")


25/02/24 08:33:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/02/24 08:33:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/02/24 08:33:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/02/24 08:33:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/02/24 08:33:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


### Dimensão Geolocalização

In [29]:
# Adiciona a chave substituta com UUID
dim_geolocalizacao = geolocation.select(
    "geolocation_zip_code_prefix",
    "geolocation_lat",
    "geolocation_lng",
    "geolocation_city",
    "geolocation_state"
).withColumn("geolocation_sk", expr("uuid()"))  # Adiciona UUID como chave substituta

# Reordena as colunas para colocar a chave substituta no início
dim_geolocalizacao = dim_geolocalizacao.select(
    "geolocation_sk",
    "geolocation_zip_code_prefix",
    "geolocation_lat",
    "geolocation_lng",
    "geolocation_city",
    "geolocation_state"
)

# Salva a dimensão geolocalização na camada Gold
dim_geolocalizacao.write.parquet(f"{camada_destino}/dim_geolocalizacao", mode="overwrite")


25/02/24 08:33:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/02/24 08:33:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/02/24 08:33:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/02/24 08:33:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/02/24 08:33:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/02/24 08:33:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/02/24 08:33:05 WARN MemoryManager: Total allocation exceeds 95.

### Fato Pedido

In [35]:
# Join entre orders, order_items, order_payments e order_reviews
fato_pedidos = orders \
    .join(order_items, "order_id", "left") \
    .join(order_payments, "order_id", "left") \
    .join(order_reviews, "order_id", "left") \
    .groupBy(
        "order_id",
        "customer_id",
        "seller_id",  # Incluído seller_id no groupBy
        "product_id",  # Incluído product_id no groupBy
        "order_status",
        "order_purchase_timestamp",
        "order_approved_at",
        "order_delivered_carrier_date",
        "order_delivered_customer_date",
        "order_estimated_delivery_date"
    ) \
    .agg(
        sum("price").alias("total_price"),
        sum("freight_value").alias("total_freight_value"),
        count("order_item_id").alias("product_count"),
        avg("review_score").alias("avg_review_score"),
        sum("payment_value").alias("total_payment_value"),
        max("payment_installments").alias("max_payment_installments")
    )

# Join com as tabelas de dimensões para obter as chaves substitutas
fato_pedidos = fato_pedidos \
    .join(dim_cliente.select("customer_id", "customer_sk"), "customer_id", "left") \
    .join(dim_vendedor.select("seller_id", "seller_sk"), "seller_id", "left") \
    .join(dim_produto.select("product_id", "product_sk"), "product_id", "left") \
    .join(dim_pagamento.select("order_id", "payment_sk"), "order_id", "left") \
    .join(dim_avaliacao.select("order_id", "review_sk"), "order_id", "left")

# Seleciona as colunas desejadas (incluindo as chaves substitutas)
fato_pedidos = fato_pedidos.select(
    "order_id",
    "customer_sk",
    "seller_sk",
    "product_sk",
    "payment_sk",
    "review_sk",
    "order_status",
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date",
    "total_price",
    "total_freight_value",
    "product_count",
    "avg_review_score",
    "total_payment_value",
    "max_payment_installments"
)

# Salva a tabela fato atualizada
fato_pedidos.write.parquet(f"{camada_destino}/fato_pedidos", mode="overwrite")

25/02/24 08:34:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/02/24 08:34:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/02/24 08:34:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/02/24 08:34:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/02/24 08:34:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/02/24 08:34:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/02/24 08:34:36 WARN MemoryManager: Total allocation exceeds 95.